In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import datasets


In [2]:
FEATURE_SEPAL_LENGTH = 'SepalLength'
FEATURE_SEPAL_WIDTH = 'SepalWidth'
FEATURE_PETAL_LENGTH = 'PetalLength'
FEATURE_PETAL_WIDTH = 'PetalWidth'
LABEL = 'label'


In [3]:
# load the data set
iris = datasets.load_iris()

iris_data_w_target = [];

# add the target to the data
for i in range(len(iris.data)):
    value = np.append(iris.data[i], iris.target[i])
    iris_data_w_target.append(value)


In [4]:
columns_names = [FEATURE_SEPAL_LENGTH, FEATURE_SEPAL_WIDTH, FEATURE_PETAL_LENGTH, FEATURE_PETAL_WIDTH, LABEL]

df = pd.DataFrame(data=iris_data_w_target, columns=columns_names)

# shuffle our data
df = df.sample(frac=1).reset_index(drop=True)


In [5]:
# divide data into training and testing data (80%/20%)
test_len = (len(df) * 20) // 100;
training_df = df[test_len:]
test_df = df[:test_len]


In [6]:
iris_feature_columns = [
    tf.contrib.layers.real_valued_column(FEATURE_SEPAL_LENGTH, dimension=1, dtype=tf.float32),
    tf.contrib.layers.real_valued_column(FEATURE_SEPAL_WIDTH, dimension=1, dtype=tf.float32),
    tf.contrib.layers.real_valued_column(FEATURE_PETAL_LENGTH, dimension=1, dtype=tf.float32),
    tf.contrib.layers.real_valued_column(FEATURE_PETAL_WIDTH, dimension=1, dtype=tf.float32)
]

x = {
    FEATURE_SEPAL_LENGTH: np.array(training_df[FEATURE_SEPAL_LENGTH]),
    FEATURE_SEPAL_WIDTH: np.array(training_df[FEATURE_SEPAL_WIDTH]),
    FEATURE_PETAL_LENGTH: np.array(training_df[FEATURE_PETAL_LENGTH]),
    FEATURE_PETAL_WIDTH: np.array(training_df[FEATURE_PETAL_WIDTH])
}


In [7]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=iris_feature_columns,
    hidden_units=[5, 5],
    n_classes=3)

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x,
    y=np.array(training_df[LABEL]).astype(int),
    num_epochs=None,
    shuffle=True)

# Train model.
classifier.train(input_fn=train_input_fn, steps=4000)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\camp-pry\\AppData\\Local\\Temp\\tmpm6xq0io5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000274FAB45C18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 

In [8]:
x = {
    FEATURE_SEPAL_LENGTH: np.array(test_df[FEATURE_SEPAL_LENGTH]),
    FEATURE_SEPAL_WIDTH: np.array(test_df[FEATURE_SEPAL_WIDTH]),
    FEATURE_PETAL_LENGTH: np.array(test_df[FEATURE_PETAL_LENGTH]),
    FEATURE_PETAL_WIDTH: np.array(test_df[FEATURE_PETAL_WIDTH])
}

# Define the training inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x,
    y=np.array(test_df[LABEL]).astype(int),
    num_epochs=1,
    shuffle=False)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("Test Accuracy: ", accuracy_score)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-27-07:16:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\camp-pry\AppData\Local\Temp\tmpm6xq0io5\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-27-07:16:29
INFO:tensorflow:Saving dict for global step 4000: accuracy = 0.96666664, average_loss = 0.062627286, global_step = 4000, loss = 1.8788185
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4000: C:\Users\camp-pry\AppData\Local\Temp\tmpm6xq0io5\model.ckpt-4000
Test Accuracy:  0.96666664


In [9]:
x = {
    FEATURE_SEPAL_LENGTH: np.array([5.0, 6.7, 7.4]),
    FEATURE_SEPAL_WIDTH: np.array([3.5, 3.1, 2.8]),
    FEATURE_PETAL_LENGTH: np.array([1.3, 4.4, 6.1]),
    FEATURE_PETAL_WIDTH: np.array([0.3, 1.4, 1.9])
}

expected = np.array([0, 1, 2])

# Define the training inputs
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x,
    num_epochs=1,
    shuffle=False)

predictions = classifier.predict(input_fn=predict_input_fn)

for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('\nPrediction is "{}" (certainity {:.1f}%), expected "{}"'.format(class_id, 100 * probability, expec))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\camp-pry\AppData\Local\Temp\tmpm6xq0io5\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "0" (certainity 97.9%), expected "0"

Prediction is "1" (certainity 99.9%), expected "1"

Prediction is "2" (certainity 99.9%), expected "2"


In [10]:
def serving_input_receiver_fn():
    serialized_tf_example = tf.placeholder(dtype=tf.string, shape=[None], name='input_tensors')
    receiver_tensors = {'predictor_inputs': serialized_tf_example}
    feature_spec = {FEATURE_SEPAL_LENGTH: tf.FixedLenFeature([25], tf.float32),
                    FEATURE_SEPAL_WIDTH: tf.FixedLenFeature([25], tf.float32),
                    FEATURE_PETAL_LENGTH: tf.FixedLenFeature([25], tf.float32),
                    FEATURE_PETAL_WIDTH: tf.FixedLenFeature([25], tf.float32)}
    features = tf.parse_example(serialized_tf_example, feature_spec)
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)


model_dir = classifier.export_savedmodel(export_dir_base="stored_model",
                                         serving_input_receiver_fn=serving_input_receiver_fn,
                                         as_text=True)
print('Model exported to ' + model_dir.decode())


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from C:\Users\camp-pry\AppData\Local\Temp\tmpm6xq0io5\model.ckpt-4000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: stored_model\temp-b'1532676921'\saved_model.pbtxt
Model exported to stored_model\1532676921
